# 🧭 QAOA para o Problema do Caixeiro Viajante (TSP)

## Implementação Didática com Execução no IBM Quantum

Este notebook implementa o TSP usando QAOA seguindo a formulação matemática:

1. **Variáveis de decisão**: $x_{i,t} \in \{0,1\}$
2. **Formulação QUBO**: $H_{QUBO} = H_{dist} + H_{p1} + H_{p2}$
3. **Mapeamento quântico**: $x_{i,t} \rightarrow \frac{I - \hat{Z}_{i,t}}{2}$
4. **QAOA**: Operadores $U_C(\gamma)$ e $U_M(\beta)$

### Comparações Realizadas
- **Brute Force** (clássico ótimo)
- **QAOA Simulador** (Aer - sem ruído)
- **QAOA IBM Quantum** (hardware real - com ruído)

---
## 📦 Instalação de Dependências (Google Colab)

In [ ]:
# Instalar pacotes necessários no Colab
!pip install qiskit>=1.0 -q
!pip install qiskit-aer -q
!pip install qiskit-ibm-runtime -q
!pip install pylatexenc -q

print("✅ Pacotes instalados com sucesso!")

In [ ]:
# Imports
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import numpy as np
from scipy.optimize import minimize
from itertools import permutations
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# Simulador local
sim_local = AerSimulator()

print("✅ Bibliotecas importadas com sucesso!")

---
## 🔑 Configuração do IBM Quantum Cloud

In [ ]:
# =============================================================================
# CREDENCIAIS IBM QUANTUM (IBM CLOUD)
# Execute apenas UMA VEZ para salvar as credenciais
# =============================================================================

QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token="xxx",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b58cd9ceef164e4586608dca729daacc:6117674a-f3e7-4bc3-8062-30e14b62d3be::",
    overwrite=True
)

print("✅ Credenciais salvas com sucesso!")

In [ ]:
# =============================================================================
# CONECTAR AO IBM QUANTUM
# =============================================================================

service = QiskitRuntimeService(
    channel="ibm_cloud", 
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/b58cd9ceef164e4586608dca729daacc:6117674a-f3e7-4bc3-8062-30e14b62d3be::"
)

print("✅ Conectado ao IBM Quantum!")

# Listar backends disponíveis
print("\n📡 Backends disponíveis na sua conta:")
print("-" * 60)
backends = service.backends()
for b in backends:
    status = "🟢" if b.status().operational else "🔴"
    print(f"   {status} {b.name}: {b.num_qubits} qubits")

In [ ]:
# =============================================================================
# SELECIONAR BACKEND IBM QUANTUM
# =============================================================================

backend_ibm = service.backend("ibm_fez")

print(f"\n🎯 Backend selecionado: {backend_ibm.name}")
print(f"   Qubits disponíveis: {backend_ibm.num_qubits}")
print(f"   Status: {'Operacional ✅' if backend_ibm.status().operational else 'Offline ❌'}")

---
## 📊 Definição do Problema TSP

### Variáveis de Decisão $x_{i,t}$

$$x_{i,t} = \begin{cases} 1 & \text{se a cidade } i \text{ é visitada no passo } t \\ 0 & \text{caso contrário} \end{cases}$$

Para $n$ cidades, temos $n^2$ variáveis binárias, ou seja, **$n^2$ qubits**.

In [ ]:
# Matrizes de distâncias
graphs = {
    3: np.array([
        [0, 10, 15],
        [10, 0, 20],
        [15, 20, 0]
    ], dtype=float),
    
    4: np.array([
        [0, 1, 50, 50],
        [1, 0, 2, 50],
        [50, 2, 0, 3],
        [50, 50, 3, 0]
    ], dtype=float),
    
    5: np.array([
        [0, 2, 9, 10, 7],
        [1, 0, 6, 4, 3],
        [15, 7, 0, 8, 3],
        [6, 3, 12, 0, 11],
        [9, 7, 5, 6, 0]
    ], dtype=float),
    
    6: np.array([
        [0, 3, 6, 7, 8, 9],
        [3, 0, 5, 6, 7, 8],
        [6, 5, 0, 4, 5, 6],
        [7, 6, 4, 0, 3, 4],
        [8, 7, 5, 3, 0, 2],
        [9, 8, 6, 4, 2, 0]
    ], dtype=float)
}

# Matrizes para IBM Quantum (apenas 3 e 4 cidades)
graphs_ibm = {k: v for k, v in graphs.items() if k <= 4}

for n, D in graphs.items():
    print(f"\n📍 Matriz D ({n} cidades) - {n**2} qubits:")
    print(D.astype(int))

In [ ]:
def qubit_index(cidade, tempo, n):
    """Mapeia (cidade, tempo) → índice do qubit."""
    return cidade * n + tempo

# Visualização do mapeamento
print("🔢 Mapeamento x_{i,t} → qubit (n=3):")
print("="*50)
for i in range(3):
    for t in range(3):
        q = qubit_index(i, t, 3)
        print(f"x_{{{i},{t}}} → q[{q}]   Cidade {i} no tempo {t}")

---
## 📐 Construção do Hamiltoniano de Custo

$$H_{QUBO} = H_{dist} + A \cdot H_{p1} + A \cdot H_{p2}$$

Mapeamento: $x_{i,t} \rightarrow \frac{I - Z_{i,t}}{2}$

In [ ]:
def construir_hamiltoniano_tsp(D, penalty_multiplier=2.0):
    """Constrói coeficientes h (Z_i) e J (Z_i Z_j) do Hamiltoniano."""
    n = len(D)
    A = penalty_multiplier * np.max(D) * n
    
    h = {q: 0.0 for q in range(n * n)}
    J = {}
    
    # H_dist
    for i in range(n):
        for j in range(n):
            if i != j:
                d_ij = D[i, j]
                for t in range(n):
                    t_next = (t + 1) % n
                    q_a = qubit_index(i, t, n)
                    q_b = qubit_index(j, t_next, n)
                    h[q_a] -= d_ij / 4
                    h[q_b] -= d_ij / 4
                    key = (min(q_a, q_b), max(q_a, q_b))
                    J[key] = J.get(key, 0) + d_ij / 4
    
    # H_p1: cada cidade uma vez
    for i in range(n):
        for t in range(n):
            q = qubit_index(i, t, n)
            h[q] += A / 2
            for t2 in range(t + 1, n):
                q2 = qubit_index(i, t2, n)
                h[q] -= A / 2
                h[q2] -= A / 2
                key = (min(q, q2), max(q, q2))
                J[key] = J.get(key, 0) + A / 2
    
    # H_p2: cada tempo uma cidade
    for t in range(n):
        for i in range(n):
            q = qubit_index(i, t, n)
            h[q] += A / 2
            for i2 in range(i + 1, n):
                q2 = qubit_index(i2, t, n)
                h[q] -= A / 2
                h[q2] -= A / 2
                key = (min(q, q2), max(q, q2))
                J[key] = J.get(key, 0) + A / 2
    
    return h, J, A

h_demo, J_demo, A_demo = construir_hamiltoniano_tsp(graphs[3])
print(f"📐 Hamiltoniano (n=3): A={A_demo}, termos Z={len(h_demo)}, termos ZZ={len(J_demo)}")

---
## 🔄 Circuito QAOA

$U_C(\gamma) = e^{-i\gamma H_C}$ e $U_M(\beta) = e^{-i\beta \sum X_k}$

In [ ]:
def qaoa_layer(qc, h, J, gamma, beta):
    """Uma camada do QAOA."""
    # U_C(γ)
    for qubit, coef in h.items():
        if abs(coef) > 1e-10:
            qc.rz(2 * gamma * coef, qubit)
    for (q_i, q_j), coef in J.items():
        if abs(coef) > 1e-10:
            qc.cx(q_i, q_j)
            qc.rz(2 * gamma * coef, q_j)
            qc.cx(q_i, q_j)
    # U_M(β)
    for q in range(qc.num_qubits):
        qc.rx(2 * beta, q)

def qaoa_circuit(h, J, num_qubits, gammas, betas):
    """Circuito QAOA completo."""
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    for gamma, beta in zip(gammas, betas):
        qaoa_layer(qc, h, J, gamma, beta)
    return qc

# Visualizar
qc_ex = qaoa_circuit(h_demo, J_demo, 9, [0.5], [0.5])
print(f"📊 Circuito (n=3, p=1): {qc_ex.num_qubits} qubits, profundidade {qc_ex.depth()}")
qc_ex.draw('mpl', fold=80)

---
## 💰 Funções de Custo e Decodificação

In [ ]:
def decodificar_bitstring(bitstring, n):
    """Decodifica bitstring em rota TSP."""
    x = np.zeros((n, n), dtype=int)
    for idx, bit in enumerate(bitstring):
        x[idx // n, idx % n] = int(bit)
    
    for i in range(n):
        if np.sum(x[i, :]) != 1:
            return None, False
    for t in range(n):
        if np.sum(x[:, t]) != 1:
            return None, False
    
    rota = [np.where(x[:, t] == 1)[0][0] for t in range(n)]
    rota.append(rota[0])
    return tuple(rota), True

def tsp_cost(bitstring, D):
    """Calcula custo da solução."""
    n = len(D)
    rota, valida = decodificar_bitstring(bitstring, n)
    if not valida:
        return float('inf'), None, False
    custo = sum(D[rota[t], rota[t + 1]] for t in range(n))
    return custo, rota, True

def processar_counts(counts, D):
    """Processa contagens de medição."""
    n = len(D)
    total = sum(counts.values())
    penalidade = np.max(D) * n * 10
    
    exp_cost, n_validas = 0, 0
    melhor_custo, melhor_rota = float('inf'), None
    
    for bs, count in counts.items():
        bs_clean = bs.replace(" ", "")[::-1]
        custo, rota, valida = tsp_cost(bs_clean, D)
        if valida:
            n_validas += count
            exp_cost += count * custo
            if custo < melhor_custo:
                melhor_custo, melhor_rota = custo, rota
        else:
            exp_cost += count * penalidade
    
    return {'exp_cost': exp_cost/total, 'frac_validas': n_validas/total,
            'melhor_rota': melhor_rota, 'melhor_custo': melhor_custo}

def brute_force_tsp(D):
    """Solução ótima por força bruta."""
    n = len(D)
    melhor_rota, melhor_custo = None, float('inf')
    for perm in permutations(range(1, n)):
        rota = (0,) + perm + (0,)
        custo = sum(D[rota[i], rota[i+1]] for i in range(n))
        if custo < melhor_custo:
            melhor_custo, melhor_rota = custo, rota
    return melhor_rota, melhor_custo

---
## 🎯 Otimização com COBYLA

In [ ]:
def objective_sim(params, h, J, D, num_qubits, p, shots=2048):
    """Função objetivo (simulador)."""
    qc = qaoa_circuit(h, J, num_qubits, params[:p], params[p:])
    qc.measure_all()
    tqc = transpile(qc, sim_local)
    counts = sim_local.run(tqc, shots=shots).result().get_counts()
    return processar_counts(counts, D)['exp_cost']

def otimizar_parametros(D, p=2, maxiter=200):
    """Otimiza γ e β."""
    n = len(D)
    h, J, A = construir_hamiltoniano_tsp(D)
    init = np.random.uniform(0, np.pi, 2 * p)
    res = minimize(objective_sim, init, args=(h, J, D, n*n, p),
                   method="COBYLA", options={'maxiter': maxiter})
    return res.x[:p], res.x[p:], h, J, A

---
## 🚀 Execução Completa

In [ ]:
PARAMS = {'p': 3, 'shots': 8192, 'maxiter': 300}
print("⚙️ Parâmetros:", PARAMS)

In [ ]:
def executar_comparacao_completa(graphs_sim, graphs_ibm, backend_ibm, params):
    """Executa BF + Simulador + IBM Quantum."""
    resultados = []
    parametros_otimizados = {}
    
    # FASE 1: BRUTE FORCE + SIMULADOR
    print("\n" + "="*70)
    print("📊 FASE 1: BRUTE FORCE + QAOA SIMULADOR")
    print("="*70)
    
    for n_cidades, D in graphs_sim.items():
        print(f"\n{'='*60}")
        print(f"📍 {n_cidades} CIDADES ({n_cidades**2} qubits)")
        print(f"{'='*60}")
        
        n = len(D)
        num_qubits = n ** 2
        
        # Brute Force
        print(f"\n[1/2] 🔍 Brute Force...")
        t0 = time.time()
        rota_bf, custo_bf = brute_force_tsp(D)
        tempo_bf = time.time() - t0
        print(f"      ✅ Rota: {rota_bf}, Custo: {custo_bf}, Tempo: {tempo_bf:.6f}s")
        
        # QAOA Simulador
        print(f"\n[2/2] ⚛️ QAOA Simulador (p={params['p']})...")
        t0 = time.time()
        gammas, betas, h, J, A = otimizar_parametros(D, p=params['p'], maxiter=params['maxiter'])
        
        qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
        qc.measure_all()
        tqc = transpile(qc, sim_local)
        counts = sim_local.run(tqc, shots=params['shots']).result().get_counts()
        tempo_sim = time.time() - t0
        
        res_sim = processar_counts(counts, D)
        gap_sim = ((res_sim['melhor_custo'] - custo_bf) / custo_bf * 100) if res_sim['melhor_rota'] else float('inf')
        
        print(f"      ✅ Rota: {res_sim['melhor_rota']}")
        print(f"      ✅ Custo: {res_sim['melhor_custo']}")
        print(f"      ✅ Válidas: {100*res_sim['frac_validas']:.2f}%")
        print(f"      ✅ Gap: {gap_sim:.2f}%")
        print(f"      ✅ Tempo: {tempo_sim:.2f}s")
        
        if n_cidades in graphs_ibm:
            parametros_otimizados[n_cidades] = {'gammas': gammas, 'betas': betas, 'h': h, 'J': J, 'D': D}
        
        resultados.append({
            'Cidades': n_cidades, 'Qubits': num_qubits,
            'Rota BF': str(rota_bf), 'Custo BF': custo_bf, 'Tempo BF (s)': tempo_bf,
            'Rota Sim': str(res_sim['melhor_rota']),
            'Custo Sim': res_sim['melhor_custo'] if res_sim['melhor_custo'] != float('inf') else 'N/A',
            'Gap Sim (%)': gap_sim if gap_sim != float('inf') else 'N/A',
            'Válidas Sim (%)': res_sim['frac_validas'] * 100,
            'Tempo Sim (s)': tempo_sim,
            'Rota IBM': None, 'Custo IBM': None, 'Gap IBM (%)': None,
            'Válidas IBM (%)': None, 'Tempo IBM (s)': None
        })
    
    # FASE 2: IBM QUANTUM
    print("\n" + "="*70)
    print(f"☁️ FASE 2: QAOA IBM QUANTUM ({backend_ibm.name})")
    print("="*70)
    
    # Criar Sampler com backend (nova API)
    sampler = Sampler(backend=backend_ibm)
    
    for n_cidades, p_opt in parametros_otimizados.items():
        print(f"\n📍 IBM Quantum: {n_cidades} cidades ({n_cidades**2} qubits)...")
        
        D = p_opt['D']
        h, J = p_opt['h'], p_opt['J']
        gammas, betas = p_opt['gammas'], p_opt['betas']
        num_qubits = n_cidades ** 2
        
        qc = qaoa_circuit(h, J, num_qubits, gammas, betas)
        qc.measure_all()
        
        print(f"   🔧 Transpilando...")
        qc_t = transpile(qc, backend=backend_ibm, optimization_level=3)
        print(f"      Profundidade: {qc_t.depth()}")
        
        print(f"   🚀 Enviando job...")
        t0 = time.time()
        job = sampler.run([qc_t], shots=params['shots'])
        print(f"      Job ID: {job.job_id()}")
        print(f"   ⏳ Aguardando...")
        
        result = job.result()
        tempo_ibm = time.time() - t0
        
        counts = result[0].data.meas.get_counts()
        res_ibm = processar_counts(counts, D)
        
        rota_bf, custo_bf = brute_force_tsp(D)
        gap_ibm = ((res_ibm['melhor_custo'] - custo_bf) / custo_bf * 100) if res_ibm['melhor_rota'] else float('inf')
        
        print(f"   ✅ Rota: {res_ibm['melhor_rota']}")
        print(f"   ✅ Custo: {res_ibm['melhor_custo']}")
        print(f"   ✅ Válidas: {100*res_ibm['frac_validas']:.2f}%")
        print(f"   ✅ Gap: {gap_ibm:.2f}%")
        print(f"   ✅ Tempo: {tempo_ibm:.2f}s")
        
        for r in resultados:
            if r['Cidades'] == n_cidades:
                r['Rota IBM'] = str(res_ibm['melhor_rota'])
                r['Custo IBM'] = res_ibm['melhor_custo'] if res_ibm['melhor_custo'] != float('inf') else 'N/A'
                r['Gap IBM (%)'] = gap_ibm if gap_ibm != float('inf') else 'N/A'
                r['Válidas IBM (%)'] = res_ibm['frac_validas'] * 100
                r['Tempo IBM (s)'] = tempo_ibm
                break
    
    return pd.DataFrame(resultados)

In [ ]:
# EXECUTAR
print("="*70)
print("🚀 COMPARAÇÃO: BRUTE FORCE vs SIMULADOR vs IBM QUANTUM")
print("="*70)

df_resultados = executar_comparacao_completa(graphs, graphs_ibm, backend_ibm, PARAMS)

print(f"\n{'='*70}")
print("✅ PROCESSAMENTO CONCLUÍDO!")
print("="*70)

---
## 📊 Resultados

In [ ]:
print("\n📊 TABELA COMPLETA")
display(df_resultados)

In [ ]:
# Tabela resumida
df_resumo = df_resultados[[
    'Cidades', 'Qubits', 'Tempo BF (s)', 'Tempo Sim (s)', 'Tempo IBM (s)',
    'Custo BF', 'Custo Sim', 'Custo IBM', 'Gap Sim (%)', 'Gap IBM (%)',
    'Válidas Sim (%)', 'Válidas IBM (%)'
]]
print("\n📋 RESUMO")
display(df_resumo)

In [ ]:
# Gráficos
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
x = df_resultados['Cidades'].values
width = 0.25

# Tempos
ax1 = axes[0, 0]
t_ibm = [t if t else 0 for t in df_resultados['Tempo IBM (s)']]
ax1.bar(x - width, df_resultados['Tempo BF (s)'], width, label='BF', color='green')
ax1.bar(x, df_resultados['Tempo Sim (s)'], width, label='Sim', color='steelblue')
ax1.bar(x + width, t_ibm, width, label='IBM', color='purple')
ax1.set_xlabel('Cidades'); ax1.set_ylabel('Tempo (s)'); ax1.set_title('Tempo')
ax1.legend(); ax1.set_xticks(x); ax1.set_yscale('log')

# Custos
ax2 = axes[0, 1]
c_sim = [c if c != 'N/A' else 0 for c in df_resultados['Custo Sim']]
c_ibm = [c if c not in ['N/A', None] else 0 for c in df_resultados['Custo IBM']]
ax2.bar(x - width, df_resultados['Custo BF'], width, label='BF', color='green')
ax2.bar(x, c_sim, width, label='Sim', color='steelblue')
ax2.bar(x + width, c_ibm, width, label='IBM', color='purple')
ax2.set_xlabel('Cidades'); ax2.set_ylabel('Custo'); ax2.set_title('Custo')
ax2.legend(); ax2.set_xticks(x)

# Gap
ax3 = axes[1, 0]
g_sim = [g if g not in ['N/A', float('inf')] else 100 for g in df_resultados['Gap Sim (%)']]
g_ibm = [g if g not in ['N/A', None, float('inf')] else 0 for g in df_resultados['Gap IBM (%)']]
ax3.bar(x - width/2, g_sim, width, label='Sim', color='steelblue')
ax3.bar(x[:len(graphs_ibm)] + width/2, g_ibm[:len(graphs_ibm)], width, label='IBM', color='purple')
ax3.set_xlabel('Cidades'); ax3.set_ylabel('Gap (%)'); ax3.set_title('Gap ao Ótimo')
ax3.axhline(y=0, color='green', linestyle='--', alpha=0.5)
ax3.legend(); ax3.set_xticks(x)

# Válidas
ax4 = axes[1, 1]
v_ibm = [v if v else 0 for v in df_resultados['Válidas IBM (%)']]
ax4.bar(x - width/2, df_resultados['Válidas Sim (%)'], width, label='Sim', color='steelblue')
ax4.bar(x + width/2, v_ibm, width, label='IBM', color='purple')
ax4.set_xlabel('Cidades'); ax4.set_ylabel('Válidas (%)'); ax4.set_title('Soluções Válidas')
ax4.legend(); ax4.set_xticks(x)

plt.tight_layout()
plt.savefig('comparacao_tsp_qaoa.png', dpi=150)
plt.show()

In [ ]:
# Análise
print("\n" + "="*70)
print("📈 ANÁLISE")
print("="*70)

print("\n🔹 BRUTE FORCE: Sempre ótimo, O(n!)")
print(f"\n🔹 SIMULADOR: Média válidas = {df_resultados['Válidas Sim (%)'].mean():.2f}%")

df_ibm_v = df_resultados[df_resultados['Válidas IBM (%)'].notna()]
if len(df_ibm_v) > 0:
    print(f"\n🔹 IBM QUANTUM ({backend_ibm.name}): Média válidas = {df_ibm_v['Válidas IBM (%)'].mean():.2f}%")
    
print("\n🔹 CONCLUSÕES:")
print("   • Hardware real tem degradação vs simulador")
print("   • QAOA funciona para n≤4 em hardware atual")
print("   • BF é superior para n pequeno")

In [ ]:
df_resultados.to_csv('resultados_tsp.csv', index=False)
print("📁 Resultados salvos em 'resultados_tsp.csv'")